In [36]:
import os

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm


from utils import api, endpoints

import pandas as pd


In [32]:
def parallel_execution(executable, iterator, max_workers = 10):
    final = []
    pbar = tqdm(range(0, len(iterator)))
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(executable, x) for x in iterator]
        for future in as_completed(futures):
            final += future.result()
            pbar.update(1)
    pbar.close()
    return final

def get_legislatura_lideres(id_):
    url = api.construct_url(api.HOST, endpoints.LEGISLATURA_LIDERES.format(id=id_))
    resp = api.request(url)
    return resp


def get_legislatura_mesa(id_):
    url = api.construct_url(api.HOST, endpoints.LEGISLATURA_MESA.format(id=id_))
    resp = api.request(url)
    return resp


def get_parlamentares_por_legislatura(id_legis):
    url = api.construct_url(api.HOST,endpoints.PARLAMENTARES, {'idLegislatura': id_legis})
    parls = api.request(url)
    return parls

def get_parlamentare_detalhes(id_parl):
    url = api.construct_url(api.HOST, endpoints.PARLAMENTARES_DETALHES.format(id_parl=id_parl))
    resp = api.request(url)
    return resp

def get_parlamentare_mandatos_externos(id_parl):
    url = api.construct_url(api.HOST, endpoints.PARLAMENTARES_MANDATOS_EXTERNOS.format(id_parl=id_parl))
    resp = api.request(url)
    final = [{"id_parl": id_parl, **r} for r in resp]
    return final

def get_parlamentare_historico(id_parl):
    url = api.construct_url(api.HOST, endpoints.PARLAMENTARES_HISTORICO.format(id_parl=id_parl))
    resp = api.request(url)
    return resp

In [20]:
LEGISLATURAS = range(48,58)

## 1.1) Parlamentares

In [12]:
parlamentares = parallel_execution(get_parlamentares_por_legislatura, LEGISLATURAS)
df_parlamentares = pd.DataFrame(parlamentares)
df_parlamentares.to_excel('./data/df_parlamentares_por_legislatura.xlsx', index=False)

100%|██████████| 10/10 [00:14<00:00,  1.40s/it]


In [34]:
ids = df_parlamentares['id'].unique()

## 1.1.1) parlamentares detalhes

In [16]:
parl_detalhes = parallel_execution(get_parlamentare_detalhes, ids)
df_parl_detalhes = pd.DataFrame(parl_detalhes)
df_parl_detalhes.to_excel('./data/df_parl_detalhes.xlsx')

## 1.2) Líderes

In [7]:
legis_lideres = parallel_execution(get_legislatura_lideres, LEGISLATURAS)
df_legis_lideres = pd.DataFrame(legis_lideres)
df_legis_lideres.to_excel('./data/df_legis_lideres.xlsx', index=False)

100%|██████████| 10/10 [03:41<00:00, 22.11s/it]


## 1.3) Mesa

In [33]:
legis_mesa = parallel_execution(get_legislatura_mesa, LEGISLATURAS)
df_legis_mesa = pd.DataFrame(legis_mesa)
df_legis_mesa.to_excel('./data/df_legis_mesa.xlsx', index=False)

100%|██████████| 10/10 [00:01<00:00,  5.03it/s]


# 2) Mandatos externos

In [12]:
cargos_externos = parallel_execution(get_parlamentare_mandatos_externos, ids)
df_cargos_externos = pd.DataFrame(cargos_externos)
df_cargos_externos.to_excel('./data/df_cargos_externos.xlsx', index=False)

Baixando mandatos externos dos parlamentares: 100%|██████████| 10/10 [00:01<00:00,  9.18it/s]


# 3) Afastamentos por parlamentar

In [22]:
historico = parallel_execution(get_parlamentare_historico, ids)
df_historico = pd.DataFrame(historico)
df_historico.to_excel('./data/df_historico.xlsx', index=False)

Baixando mandatos externos dos parlamentares: 100%|██████████| 10/10 [00:01<00:00,  8.01it/s]


#

# Relatorias por parlamentar